# Day 2 - Intcode interpreter

* https://adventofcode.com/2019/day/2

We have a computer again! We've seen this before in 2017 ([day 18](../2017/Day%2018.ipynb), [day 23](../2017/Day%2023.ipynb)), and 2018 ([day 16](../2018/Day%2016.ipynb), [day 19](../2018/Day%2019.ipynb) and [day 21](../2018/Day%2021.ipynb)).

Now we have opcodes with a variable number of operands (called *positions* here); `1` and `2` each have 2 operands and output destination, 99 has none. There are also no registers, all operations take place directly on the memory where our code is also stored, so it can self-modify. Fun!

So we need a CPU with a position counter, memory, and opcode definitions (*instructions*) to call, and the opcodes need access to the memory (to read operand values and write out their result to). Easy peasy.

I'm assuming we'll expand on the instruction set later on, and that we might have instructions with different numbers of operands. So given a function to process the input values and the number of *paramaters* to process, we should be able to produce something readable and reusable.

In [1]:
import aocd
data = aocd.get_data(day=2, year=2019)
memory = list(map(int, data.split(',')))

In [2]:
from __future__ import annotations

import operator
from dataclasses import dataclass
from typing import Callable, List, Mapping, Optional

Memory = List[int]

class Halt(Exception):
    """Signal to end the program"""
    @classmethod
    def halt(cls) -> int:  # yes, because Opcode.f callables always produce ints, right?
        raise cls

@dataclass
class Instruction:
    # the inputs are processed by a function that operates on integers
    # returns integers to store in a destination position
    f: Callable[..., int]
    # An opcode takes N paramaters
    paramater_count: int

    def __call__(self, memory: Memory, *parameters: int) -> None:
        if parameters:
            *inputs, output = parameters
            memory[output] = self.f(*(memory[addr] for addr in inputs))
        else:
            # no parameter count, so just call the function directly, no output expected
            self.f()


class CPU:
    memory: Memory
    pos: int
    opcodes: Mapping[int, Instruction] = {
        1: Instruction(operator.add, 3),
        2: Instruction(operator.mul, 3),
        99: Instruction(Halt.halt, 0),
    }
        
    def reset(self, memory: Memory = None):
        if memory is None:
            memory = []
        self.memory = memory[:]
        self.pos: int = 0
        
    def execute(
        self, memory: Memory,
        noun: Optional[int] = None,
        verb: Optional[int] = None
    ) -> int:
        self.reset(memory)
        memory = self.memory
        if noun is not None:
            memory[1] = noun
        if verb is not None:
            memory[2] = verb
        try:
            while True:
                op = self.opcodes[memory[self.pos]]
                paramcount = op.paramater_count
                parameters = memory[self.pos + 1 : self.pos + 1 + paramcount]
                op(memory, *parameters)
                self.pos += 1 + paramcount
        except Halt:
            return memory[0]

test: Memory = [1, 9, 10, 3, 2, 3, 11, 0, 99, 30, 40, 50]
cpu = CPU()
assert cpu.execute(test) == 3500

In [3]:
print('Part 1:', cpu.execute(memory, 12, 2))

Part 1: 3706713


## Part 2

Now we need to find the noun and verb that produce a specific programme output. The text suggests we should just brute-force this, so lets try that first and see how long that takes. Given that we'll only have to search through 10000 different inputs, and there are no options to loop, that's not that big a search space anyway.

While the code can self-modify, this is limited to:

- altering what inputs are read
- where to write the output
- replacing a read or write op with another read, write or halt op

so we execute, at most, `len(memory) // 4` instructions, which for my input means there are only 32 steps per execution run, and so we are going to execute, at most, 32.000 instructions. That's pretty cheap:

In [4]:
from itertools import product
def bruteforce(target: int, memory: Memory) -> int:
    cpu = CPU()
    for noun, verb in product(range(100), repeat=2):
        result = cpu.execute(memory, noun, verb)
        if result == target:
            break
    return 100 * noun + verb

print('Part 2:', bruteforce(19690720, memory))

Part 2: 8609


## Avoiding brute force

Can we just calculate the number? We'd have to disassemble the inputs to see what is going on. Provided the programme never alters its own instructions, we should be able to figure this out.

Lets see if we need to worry about self-modifying code first:

In [5]:
# code is expected to alter memory[0], so we don't count that
# as self modifying as the CPU will never return there.
if any(target and target % 4 == 0 for target in memory[3::4]):
    print('Code targets opcodes')
elif any(target % 4 and target > (i * 4 + 3) for i, target in enumerate(memory[3::4])):
    print('Code targets parameters of later opcodes')
else:
    print('Code is not self-modifying')

Code is not self-modifying


For my puzzle input, the above declares the code to not be self modifying. So all we have is addition and multiplication of memory
already harvested for opcodes and parameter addresses. It's just a big sum! 

Note that some operations might write to a destination address that is then never read from, or overwritten by other operations. We could just eliminate those steps, if we could detect those cases.

### What does the sum look like?

We can skip the first operation (`ADD 1 2 3`) because the *next* expression also writes to `3` without using the outcome of the first. That makes sense, because `1` and `2` are our `noun` and `verb` inputs and those can be anywhere in the programme. Or, like I do below, you can just skip the type error that `listobject[string]` throws when trying to use either `'noun'` or `'verb'` as indices.

In [6]:
fmemory = memory[:]
fmemory[1:3] = 'noun', 'verb'
for opcode, a, b, target in zip(*([iter(fmemory)] * 4)):
    if opcode == 99:
        break
    try:
        fmemory[target] = f"({fmemory[a]}{' +*'[opcode]}{fmemory[b]})"
    except TypeError as e:
        # the first instruction is to add memory[noun] and memory[verb]
        # and store in 3 but the next instruction also stores in 3,
        # ignoring the previous result.
        assert a == 'noun' and b == 'verb'

formula = fmemory[0]
print(formula)

(3+((1+(3*(1+((3+(3+(5*(2*(4+((5*(1+((5*(1+(2*((4*((((2+(5+(2+(noun*4))))+4)+2)+5))+2))))*3)))+1))))))*3))))+verb))


If you were to compile this to a function; Python's AST optimizer will actually replace a lot of the constants; I'm using [Astor](https://github.com/berkerpeksag/astor/) here to simplify roundtripping and pretty printing, so we can see what Python makes of it:

In [7]:
import ast
import astor
from textwrap import wrap

simplified = astor.to_source(ast.parse(formula))
print("19690720 =", simplified)

19690720 = 3 + (1 + 3 * (1 + (3 + (3 + 5 * (2 * (4 + (5 * (1 + 5 * (1 + 2 * (4 * (2 +
    (5 + (2 + noun * 4)) + 4 + 2 + 5) + 2)) * 3) + 1))))) * 3) + verb)



This is something we can work with! Clearly this is a simple [linear Diophantine equation](https://en.wikipedia.org/wiki/Diophantine_equation#Linear_Diophantine_equations) that can be solved for either `noun` or `verb`, so let's see if [sympy](https://docs.sympy.org/latest/), the Python symbolic maths solver can do something with this.

We know that both `noun` and `verb` are values in the range `[0, 100)`, so we can use this to see what inputs in that range produce an output in that range:

In [8]:
import dis
from IPython.display import display, Markdown
from sympy import diophantine, lambdify, symbols, sympify, Eq, Symbol
from sympy.solvers import solve

# ask Sympy to parse our formula; it'll simplify the formula for us
display(Markdown("### Simplified expression:"))
expr = sympify(formula) - 19690720
display(expr)

# extract the symbols
noun, verb = sorted(expr.free_symbols, key=lambda s: s.name)
                
display(Markdown("### Solution for the linear diophantine equation"))
# solutions for the two input variables, listed in alphabetical order,
for noun_expr, verb_expr in diophantine(expr):
    if isinstance(noun_expr, Symbol):
        solution = verb_expr.subs(noun_expr, noun)
        arg, result = noun, verb
    else:
        solution = noun_expr.subs(verb_expr, verb)
        arg, result = verb, noun.name
    display(Eq(result, solution))

for i in range(100):
    other = solution.subs(arg, i)
    if 0 <= other < 100:
        noun_value = other if result.name == 'noun' else i
        verb_value = i if result.name == 'noun' else other
        display(Markdown(
f"""### Solution found:

* $noun = {noun_value}$
* $verb = {verb_value}$
* $100noun + verb = {100 * noun_value + verb_value}$
"""))

        break


### Simplified expression:

216000*noun + verb - 18576009

### Solution for the linear diophantine equation

Eq(verb, 18576009 - 216000*noun)

### Solution found:

* $noun = 86$
* $verb = 9$
* $100noun + verb = 8609$


Unfortunately, even Sympy's `solveset()` function couldn't help me eliminate the loop over `range(100)`; in principle this should be possible using an `Range(100)` set, but `solveset()` just isn't quite there yet. A [related question on Stack Overflow](https://stackoverflow.com/questions/46013884/get-all-positive-integral-solutions-for-a-linear-equation) appears to confirm that using a loop is the correct method here. I could give Sage a try for this, perhaps.

That said, if you look at the $term1 - term2 \times arg$ solution to the diophantine equation, to me it is clear that `noun` and `verb` are simply the division and modulus, respectively, of $term1$ and $term2$:

In [9]:
from sympy import postorder_traversal, Integer
term1, term2 = (abs(int(e)) for e in postorder_traversal(expr) if isinstance(e, Integer))
print(f"divmod({term1}, {term2})")
noun, verb = divmod(term1, term2)
print(f"{noun=}, {verb=}, {100 * noun + verb=}")

divmod(18576009, 216000)
noun=86, verb=9, 100 * noun + verb=8609
